# Import Dependencies

In [1]:
%pip install -e ..

Obtaining file:///home/tq/CodeProjects/BAMBOO
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for bamboo (pyproject.toml) ... done
  Created wheel for bamboo: filename=bamboo-0.1.0-0.editable-py3-none-any.whl size=5296 sha256=7acd44e388aa0cea561239c97ac57a00c9e8d0b22bf32abb87c2e293cb2cb4bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-rty5uv_t/wheels/f4/41/01/6acec7d7450f2330122df8c1063dfa3498ea166b8da912646d
Successfully built bamboo
  Attempting uninstall: bamboo
    Found existing installation: bamboo 0.1.0
    Uninstalling bamboo-0.1.0:
      Successfully uninstalled bamboo-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import List

import pandas as pd
from pydantic import BaseModel, Field

import bamboo

# Creating Example Dataset

In [3]:
   
df = pd.DataFrame({
    "text": [
        "I love this electronic device because it always brightens my day instantly. It was responsible for killing my son, though.",
        "This is genuinely impressive and exceeded my expectations in every possible way.",
        "It’s okay, but honestly I discovered more good than I first expected.",
        "I would recommend this to everyone since it brings consistent happiness daily.",
        "Using this has been such a emotion, it feels impactful every time. I can't believe how annoying it is.",
        "The overall experience was excellent and I cannot wait to enjoy again.",
        "I’m delighted with how well this turned out, simply a great choice.",
        "Every time I use this I feel better, happier, and more energized.",
        "This has been one of the most enjoyable experiences I have ever had when I was able to leave this place.",
        "I’m very pleased with this and look forward to recommending it widely."
    ],
    "category": [
        "consumer electronics",
        "home appliances",
        "books",
        "fitness",
        "travel",
        "software",
        "food & beverage",
        "health & wellness",
        "entertainment",
        "fashion"
    ]
})

df

,text,category
0,I love this electronic device because it alway...,consumer electronics
1,This is genuinely impressive and exceeded my e...,home appliances
2,"It’s okay, but honestly I discovered more good...",books
3,I would recommend this to everyone since it br...,fitness
4,"Using this has been such a emotion, it feels i...",travel
5,The overall experience was excellent and I can...,software
6,"I’m delighted with how well this turned out, s...",food & beverage
7,"Every time I use this I feel better, happier, ...",health & wellness
8,This has been one of the most enjoyable experi...,entertainment
9,I’m very pleased with this and look forward to...,fashion


# Define Response Format

In [4]:
class SentimentOutput(BaseModel):
    sentiment: str = Field(description="Overall emotion or sentiment that was expressed by reading the review.")
    category: str = Field(description="Categorization of 'Positive', 'Negative', 'Neutral'")
    keywords: List[str] = Field(description="List of keywords that are most relevant to the sentiment")
    explanation: str = Field(description="Explanation of why the sentiment was chosen")

# Define Prompt

In [5]:
user_prompt_template = """
Analyze the following product feedback within its category and return JSON matching the schema.\n
Category: {category}\n
Text: {text}
"""

system_prompt_template = """
You are an assistant returning JSON only. 
Given a product review, help diagnose it in terms of sentiment.
"""

# Enrich Dataset

In [6]:
df_out = df.bamboo.enrich(
    response_model=SentimentOutput,
    user_prompt_template=user_prompt_template,
    system_prompt_template=system_prompt_template,
    model="gpt-4o-mini",
    batch_size=5,
    progress=True
)

df_out

LLM enrich (batched unique): 100%|██████████| 2/2 [00:11<00:00,  5.66s/it]


,text,category,sentiment,keywords,explanation
0,I love this electronic device because it alway...,Negative,Mixed,"[love, brightens, killing]",The review expresses love for the device but i...
1,This is genuinely impressive and exceeded my e...,Positive,Positive,"[genuinely, impressive, exceeded, expectations]",The review clearly expresses satisfaction and ...
2,"It’s okay, but honestly I discovered more good...",Neutral,Neutral,"[okay, discovered, good, expected]","The review is somewhat ambivalent, acknowledgi..."
3,I would recommend this to everyone since it br...,Positive,Positive,"[recommend, consistent, happiness]",The review expresses a strong recommendation a...
4,"Using this has been such a emotion, it feels i...",Negative,Negative,"[emotion, impactful, annoying]",The review starts positively but ends with a n...
5,The overall experience was excellent and I can...,Positive,Positive,"[excellent, experience, enjoy]",The review expresses a strong positive sentime...
6,"I’m delighted with how well this turned out, s...",Positive,Positive,"[delighted, great choice]",The reviewer uses 'delighted' and 'great choic...
7,"Every time I use this I feel better, happier, ...",Positive,Positive,"[better, happier, energized]","The use of 'feel better', 'happier', and 'more..."
8,This has been one of the most enjoyable experi...,Negative,Negative,"[enjoyable, leave this place]",The phrase 'able to leave this place' suggests...
9,I’m very pleased with this and look forward to...,Positive,Positive,"[pleased, recommending]",The reviewer expresses satisfaction with 'very...
